<a href="https://colab.research.google.com/github/dhanyahari07/medium_post/blob/main/zero_shot_learning_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [9]:
# Example texts and their corresponding labels
texts = [
    "This is a positive review.",
    "Negative sentiment detected in this text.",
    "The product is excellent.",
    "I don't recommend this at all."
]

labels = [
    "Positive",
    "Negative",
    "Positive",
    "Negative"
]

label_values=[1,0,1,0]

In [3]:
# Tokenizing the text and converting to sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

In [4]:
# Padding sequences for a consistent input shape
max_len = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post')


In [5]:
# Creating word embeddings using GloVe
embedding_dim = 50  # Adjust based on your GloVe embedding dimension
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, embedding_dim))

In [6]:
# Load GloVe embeddings
glove_file_path = '/content/drive/MyDrive/zero_shot_learning/glove.6B.50d.txt'  # Provide the path to your GloVe file
with open(glove_file_path, encoding='utf-8') as file:
    for line in file:
        values = line.split()
        word = values[0]
        if word in tokenizer.word_index:
            embedding_matrix[tokenizer.word_index[word]] = np.array(values[1:], dtype='float32')


In [7]:
# Define the model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1,
                    output_dim=embedding_dim,
                    weights=[embedding_matrix],
                    input_length=max_len,
                    trainable=False))  # Set trainable to True if you want to fine-tune embeddings
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))  # Binary classification, change based on your problem


In [14]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
#model.fit(padded_sequences, np.array(labels == "Positive", dtype=int), epochs=10, batch_size=1, verbose=1)
model.fit(padded_sequences,np.array(label_values), epochs=50, batch_size=1, verbose=1)


Epoch 1/50
4/4 [==============================] - 1s 6ms/step - loss: 0.3496 - accuracy: 1.0000
Epoch 2/50
4/4 [==============================] - 0s 8ms/step - loss: 0.3011 - accuracy: 1.0000
Epoch 3/50
4/4 [==============================] - 0s 7ms/step - loss: 0.2665 - accuracy: 1.0000
Epoch 4/50
4/4 [==============================] - 0s 7ms/step - loss: 0.2418 - accuracy: 1.0000
Epoch 5/50
4/4 [==============================] - 0s 6ms/step - loss: 0.2087 - accuracy: 1.0000
Epoch 6/50
4/4 [==============================] - 0s 4ms/step - loss: 0.1872 - accuracy: 1.0000
Epoch 7/50
4/4 [==============================] - 0s 4ms/step - loss: 0.1687 - accuracy: 1.0000
Epoch 8/50
4/4 [==============================] - 0s 4ms/step - loss: 0.1509 - accuracy: 1.0000
Epoch 9/50
4/4 [==============================] - 0s 5ms/step - loss: 0.1326 - accuracy: 1.0000
Epoch 10/50
4/4 [==============================] - 0s 5ms/step - loss: 0.1208 - accuracy: 1.0000
Epoch 11/50
4/4 [======================

In [16]:
# Example of zero-shot prediction on a new text
new_text = "I'm really impressed with this product."
#new_text = "I hate this product."
new_sequence = tokenizer.texts_to_sequences([new_text])
new_padded_sequence = pad_sequences(new_sequence, maxlen=max_len, padding='post')

# Make predictions
prediction = model.predict(new_padded_sequence)
predicted_label = "Positive" if prediction[0][0] > 0.5 else "Negative"

print(f"The model predicts the label for the new text as: {predicted_label}")

1/1 [==============================] - 0s 22ms/step
The model predicts the label for the new text as: Positive
